In [1]:
import pandas as pd
import numpy as np
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows as df_to_row

In [16]:
from szp_funcs import create_res, path_to_data, path_to_docs, print_df, get_fio

In [3]:
def to_double(row, f_name):
    if type(row[f_name]) is float:
        return row[f_name] 
    t = str(row[f_name]).split(',')
    if t[0] == 'nan':
        return 0.0
    return float(t[0]+'.'+t[1])

In [4]:
good_stat = [
"Работа",
"Отпуск неоплачиваемый по разрешению работодателя",
"Отпуск основной",
"Болезнь",
"Отпуск учебный оплачиваемый",
"Командировка",
"Болезнь без оплаты",
"Отсутствие по невыясненным причинам",
"Дополнительный отпуск",
"Дополнительный отпуск (неоплачиваемый)",
"Отсутствие с сохранением оплаты",
"Отпуск учебный неоплачиваемый",
"Отпуск неоплачиваемый по законодательству",
"Отпуск на санаторно-курортное лечение"
]

In [5]:
pd.read_excel(path_to_data + 'apr.xlsx')

,inn,snils,sum,job,day,type,stv,status,status_pref
0,7734408766,001-879-913 57,135113.91,Преподаватель,1,Основное место работы,"1,00000000000000000000",Работа,Работа
1,7734408766,001-896-420 39,126320.51,Преподаватель,1,Основное место работы,"1,00000000000000000000",Работа,Работа
2,7734408766,002-050-383 67,68007.79,Преподаватель,1,Основное место работы,"1,00000000000000000000",Работа,Работа
3,7734408766,002-065-271 85,76705.72,Преподаватель,1,Основное место работы,"1,00000000000000000000",Работа,Работа
4,7734408766,002-845-525 25,125265.61,Преподаватель,1,Основное место работы,"1,00000000000000000000",Работа,Работа
...,...,...,...,...,...,...,...,...,...
198966,7722069597,186-373-917 14,96600.00,Специалист по интернет-маркетингу,1,Основное место работы,"1,00000000000000000000",Работа,Работа
198967,7722069597,187-703-552 01,38146.67,Воспитатель,0,Основное место работы,"1,00000000000000000000",Работа,Отпуск учебный оплачиваемый
198968,7722069597,198-391-256 24,83680.00,Воспитатель,1,Основное место работы,"1,00000000000000000000",Работа,Работа
198969,7722069597,202-693-820 52,84515.24,Учитель,1,Основное место работы,"1,00000000000000000000",Работа,Работа


In [6]:
def load_data_all(file):
    df = pd.read_excel(path_to_data + file + '.xlsx')
    df.stv = df.apply(lambda row: to_double(row, 'stv'), axis=1)
    sample = df[~(df.type == 'Внешнее совместительство') & (df.status.isin(good_stat))]
    sums = sample.iloc[:, [0,1,2,3]].groupby(['inn', 'snils']).sum().reset_index().groupby('snils').max().reset_index()
    sums = pd.merge(sums, sample[sample.type == 'Основное место работы'], how='left', on=['inn', 'snils'])
    sums = sums[['inn', 'snils', 'sum_x', 'stv', 'job']].drop_duplicates(['inn', 'snils', 'sum_x'])
    sums = sums.rename(columns = {'inn': 'inn', 'sum_x': 'sum_' + file, 'job':'job_' + file, 'stv': 'stv_' + file})
    return sums


def create_full_res(months):
    res = 0
    for i in range(len(months)):
        if i == 0:
            res = load_data_all(months[i])
        else:
            res = pd.merge(res, load_data_all(months[i]), how='outer', on=['inn', 'snils'])
    return res

In [7]:
months = ['apr']

In [8]:
inns = [
7743085712,
7723920242,
7714805691,
7719286023,
7716237684,
7723356160,
7736110982,
7716081388,
7731281046,
7719286104,
7743085720,
7713385719,
7708044657,
7731281039,
7725247640,
7708235066,
7718261509,
7705513678,
7705513734,
7716237691,
7713385726,
7709438340,
7725247752,
7736000370,
7737057354,
7733020995,
7701050524,
7724035635,
7733023121,
7716079082,
7725281915,
7729434070,
7725031305,
7727270740,
7725247657,
7721222323,
7733121993,
7735128049,
7743937357,
7715398103,
7733536758,
7737132026,
7734408766,
7718261587,
7724295383,
7743085744,
7737014368,
7719285936,
7737066359
]

In [9]:
jobs = ['Преподаватель', 'Мастер производственного обучения']

In [10]:
res = create_full_res(months)

In [35]:
res

,inn,snils,sum_apr,stv_apr,job_apr
0,7720258623,001-005-586 64,114968.88,1.0,Заведующий хозяйством
1,7718309983,001-005-786 70,72633.85,1.0,Учитель
2,7735601120,001-008-669 78,39898.73,1.0,Учитель
3,7736614860,001-010-312 25,147344.30,1.0,Учитель
4,7723169690,001-011-996 67,156752.15,1.0,Учитель
...,...,...,...,...,...
162577,7727191008,215-303-665 25,47243.64,1.0,Уборщик территории
162578,7733113047,215-317-586 48,54216.28,1.0,Уборщик служебных и производственных помещений
162579,7734233259,215-465-719 66,50969.40,1.0,Уборщик территории
162580,7734233259,215-573-458 67,19127.26,1.0,Рабочий по комплексному обслуживанию и ремонту...


In [14]:
temp = res[res.inn.isin([7717043346]) & res.job_apr.isin(jobs)]

In [17]:
fio = get_fio()

In [15]:
temp

,inn,snils,sum_apr,stv_apr,job_apr
172,7717043346,001-152-819 81,142447.50,1.0,Преподаватель
488,7717043346,001-199-144 08,70869.50,1.0,Преподаватель
1347,7717043346,001-304-401 54,99232.50,1.0,Преподаватель
1349,7717043346,001-304-422 59,60955.50,1.0,Преподаватель
3834,7717043346,001-660-495 07,131148.50,1.0,Преподаватель
...,...,...,...,...,...
158565,7717043346,189-651-858 41,112563.46,1.0,Преподаватель
158663,7717043346,189-919-569 65,182040.00,1.0,Преподаватель
159397,7717043346,191-873-573 13,74353.50,1.0,Преподаватель
159592,7717043346,192-669-179 21,123874.00,1.0,Преподаватель


In [18]:
print_df(pd.merge(temp, fio, how='left', on='snils').drop_duplicates(['snils']), 'Апрель спо пед + мастер МГПУ')

In [12]:
temp = temp[['inn', 'snils', 'job_apr']].groupby(['inn', 'job_apr']).nunique()

In [13]:
temp.unstack().fillna(0.0)

,snils
job_apr,Преподаватель
inn,
7717043346,233


In [51]:
temp.unstack().fillna(0.0)

snils              
job_apr    Мастер производственного обучения Преподаватель
inn                                                       
7701050524                              16.0          73.0
7705513678                              18.0          49.0
7705513734                              25.0         221.0
7708044657                               0.0          45.0
7708235066                               9.0         128.0
7709438340                              14.0          40.0
7713385719                              31.0         122.0
7713385726                               0.0          66.0
7714805691                              10.0          89.0
7715398103                              26.0          48.0
7716079082                              15.0          81.0
7716081388                              19.0         122.0
7716237684                              32.0         227.0
7716237691                              12.0         119.0
7718261509                              14.0          63.0
7718261587                              42.0          52.0
7719285936                               9.0          44.0
7719286023                               5.0          72.0
7719286104                               3.0         165.0
7721222323                               0.0          70.0
7723356160                              69.0         281.0
7723920242                               9.0         104.0
7724035635                               5.0          68.0
7724295383                              19.0          99.0
7725031305                              11.0          94.0
7725247640                               0.0          80.0
7725247657                              58.0         137.0
7725247752                               0.0          68.0
7725281915                               6.0          56.0
7727270740                              14.0          98.0
7729434070                              15.0          68.0
7731281039                              10.0          69.0
7731281046                               3.0          44.0
7733020995                               0.0          53.0
7733023121                               4.0         147.0
7733121993                               0.0          59.0
7733536758                               0.0          47.0
7734408766                               0.0          62.0
7735128049                               5.0          66.0
7736000370                               0.0          34.0
7736110982                               0.0           6.0
7737014368                               0.0          17.0
7737057354                               1.0         151.0
7737066359                               0.0          51.0
7737132026                               0.0          53.0
7743085712                               0.0         174.0
7743085720                              15.0          98.0
7743085744                              24.0          89.0
7743937357                               7.0          94.0

In [46]:
print_df(temp.unstack().fillna(0.0), 'Апрель спо пед + мастер')